In [58]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from MysqlAdmin import ConnectionAdmin as mysql
from typing import List, Dict
import json 

In [59]:
conn = mysql.Connection('vysagota-rpm')
tables = [table['Tables_in_vysagota-rpm'] for table in conn.get("show tables")]
tables

['doctors',
 'pacients',
 'pacientsDoctors',
 'questionaryMeasures',
 'questionarys',
 'questionarysSet',
 'questionarysSetQuestionarys',
 'revisions',
 'stats']

In [60]:
table_desc = conn.get(f"describe {tables[1]}")


In [64]:

def packDesc(desc: Dict[str, str], table_name:str) -> Dict[str, str]:
    constraint = "N/A"
    constraint = constraint if desc['Null'] != 'NO' else 'NN'
    constraint = constraint if desc['Default'] == None  else desc['Default']
    constraint = constraint if desc['Key'] == '' else desc['Key']
    
    return {
        "field_name": desc["Field"],
        "constraint": constraint,
        "type": desc["Type"],
        "description": "[DESCRIPTION]",
        "size": "8",
        "table": table_name
    }
    
def getDecriptions(conn: mysql.Connection) -> List[Dict[str, str]]:
    tables = [table['Tables_in_vysagota-rpm'] for table in conn.get("show tables")]
    descriptions = []
    for table in tables:
        desc = conn.get(f"describe {table}")
        for field in desc:
            descriptions.append(packDesc(field, table))
        
    return descriptions

In [65]:
descriptions = getDecriptions(conn)

In [67]:
with open('./data_dict.json', 'w') as f:
    json.dump(descriptions, f)